In [2]:
from pandas import read_html
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib as plt
import sklearn
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import scipy.stats as scipy
import Defensive_Stats as stats
import Defense_All_Team as all_teams

In [3]:
First_Team,Second_Team = all_teams.get_all_defense()

In [4]:
First_Team.head()
First_Team.to_csv("First_Team.csv")
Second_Team.to_csv("Second_Team.csv")

In [5]:
Both_Teams = First_Team.append(Second_Team)

In [6]:
Defensive_Tracking,Defense_General,Shooting,Hustle = stats.Defensive_Stats()

In [7]:
#Player_info = stats.player_stats()
Player_info = pd.read_csv("Player_info.csv")

In [8]:
Positions = Player_info.ix[:,['Year','Player','Pos']]
Positions['Year'] = Positions['Year'].astype(float)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [9]:
def asterisk (name):
    if "*" in name:
        return name[:(len(name)-1)]
    else:
        return name

In [10]:
Positions['Player'] = Positions['Player'].apply(asterisk)

In [11]:
Defense_General = pd.merge(Defense_General,Both_Teams, on = ['Year','Player'],how = 'outer')
Defense_General = pd.merge(Defense_General,Positions, on = ['Year','Player'])
Defense_General['All_Team'] = Defense_General['All_Team'].fillna(0)
Defense_General.to_csv("Defense_General.csv")

In [12]:
Shooting = pd.merge(Shooting,Both_Teams, on = ['Year','Player'],how = 'outer')
Shooting = pd.merge(Shooting,Positions, on = ['Year','Player'])
Shooting['All_Team'] = Shooting['All_Team'].fillna(0)
Shooting.to_csv("Shooting.csv")

In [14]:
Shooting[Shooting['Player']=="Tim Duncan"]

,Player,<5Ft FGM,<5 Ft FGA,<5 Ft FG%,5-9 Ft GM,5-9 Ft FGA,5-9 Ft FG%,10-14 Ft FGM,10-14 Ft FGA,10-14 Ft FG%,...,15-19 Ft FG%,20-24 Ft FGM,20-24 Ft FGA,20-24 Ft FG%,25-29 Ft FGM,25-29 Ft FGA,25-29 Ft FG%,Year,All_Team,Pos
1049,Tim Duncan,10.5,21.5,0.490,2.9,8.4,0.349,3.7,9.7,0.384,...,0.402,3.2,8.8,0.364,1.3,4.2,0.308,1997,2.0,PF
1556,Tim Duncan,11,21.3,0.515,3.4,9.8,0.343,3.8,9.8,0.387,...,0.343,2.9,8.3,0.351,1.0,3.3,0.291,1998,1.0,PF
2046,Tim Duncan,10.6,19.8,0.538,3.5,9.4,0.374,3.3,9.6,0.343,...,0.388,3.8,9.5,0.397,1.2,4.0,0.309,1999,1.0,PF
2570,Tim Duncan,9.7,19.0,0.510,3.2,9.2,0.343,3.4,9.0,0.376,...,0.398,3.3,9.2,0.364,1.3,3.9,0.335,2000,1.0,PF
3063,Tim Duncan,11.7,22.0,0.531,3.3,8.6,0.387,3.7,9.8,0.381,...,0.377,3.8,10.0,0.383,1.3,4.2,0.299,2001,1.0,PF
3544,Tim Duncan,11.1,21.4,0.520,3.3,9.1,0.362,3.3,8.7,0.385,...,0.389,3.7,10.0,0.374,1.1,3.6,0.303,2002,1.0,PF
4096,Tim Duncan,9.6,19.4,0.494,3.0,8.5,0.353,3.0,8.3,0.362,...,0.363,2.9,8.0,0.359,1.0,3.2,0.317,2003,2.0,PF
4668,Tim Duncan,9.5,19.4,0.489,2.2,6.9,0.317,2.6,6.9,0.376,...,0.389,3.1,7.6,0.408,0.7,2.4,0.311,2004,1.0,PF
5222,Tim Duncan,11.1,21.1,0.526,2.3,6.5,0.344,2.6,7.0,0.375,...,0.383,3.1,8.0,0.390,0.9,2.6,0.358,2005,2.0,PF
5743,Tim Duncan,10.3,20.2,0.513,2.6,6.6,0.402,2.4,6.0,0.395,...,0.423,3.1,8.3,0.374,1.0,3.4,0.302,2006,1.0,C


In [13]:
PG_Defense = Defense_General[Defense_General['Pos'].str.contains("PG")]
SG_Defense = Defense_General[Defense_General['Pos'].str.contains("SG")]
SF_Defense = Defense_General[Defense_General['Pos'].str.contains("SF")]
PF_Defense = Defense_General[Defense_General['Pos'].str.contains("PF")]
C_Defense = Defense_General[Defense_General['Pos'].str.contains("C")]

In [14]:
PG_Shooting = Shooting[Shooting['Pos'].str.contains("PG")]
SG_Shooting = Shooting[Shooting['Pos'].str.contains("SG")]
SF_Shooting = Shooting[Shooting['Pos'].str.contains("SF")]
PF_Shooting = Shooting[Shooting['Pos'].str.contains("PF")]
C_Shooting = Shooting[Shooting['Pos'].str.contains("C")]

In [ ]:
First_Team = pd.merge(First_Team,Defense_General,on=['Year','Player'])
First_Team = pd.merge(First_Team,Shooting,on = ['Year','Player'])
First_Team = pd.merge(First_Team,Positions,on = ['Year','Player'])
Second_Team = pd.merge(Second_Team,Positions,on = ['Year','Player'])
Second_Team = pd.merge(Second_Team,Defense_General,on=['Year','Player'])
Second_Team = pd.merge(Second_Team,Shooting,on = ['Year','Player'])
Shooting = pd.merge(Shooting,Positions, on = ['Year','Player'])
Defense_General = pd.merge(Defense_General,Positions, on = ['Year','Player'])

Shooting.to_csv("Shooting.csv")
Defense_General.to_csv("Defense_General.csv")
First_Team.to_csv("First_Team.csv")
Second_Team.to_csv("Second_Team.csv")
Player_info.to_csv("Player_info.csv")

In [ ]:
PG_First_Team = First_Team[First_Team['Pos'].str.contains("PG")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
SG_First_Team = First_Team[First_Team['Pos'].str.contains("SG")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
SF_First_Team = First_Team[First_Team['Pos'].str.contains("SF")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
PF_First_Team = First_Team[First_Team['Pos'].str.contains("PF")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
C_First_Team = First_Team[First_Team['Pos'].str.contains("C")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
PG_Second_Team = Second_Team[Second_Team['Pos'].str.contains("PG")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
SG_Second_Team = Second_Team[Second_Team['Pos'].str.contains("SG")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
SF_Second_Team = Second_Team[Second_Team['Pos'].str.contains("SF")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
PF_Second_Team = Second_Team[Second_Team['Pos'].str.contains("PF")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]
C_Second_Team = Second_Team[Second_Team['Pos'].str.contains("C")].loc[:,['Year','Less than 5 Ft FGM',
       'Less than 5 Ft FGA', 'Less than 5 Ft FG%', '5-9 Ft GM', '5-9 Ft FGA',
       '5-9 Ft FG%', '10-14 Ft FGM', '10-14 Ft FGA', '10-14 Ft FG%',
       '15-19 Ft FGM', '15-19 Ft FGA', '15-19 Ft FG%', '20-24 Ft FGM',
       '20-24 Ft FGA', '20-24 Ft FG%', '25-29 Ft FGM', '25-29 Ft FGA',
       '25-29 Ft FG%']]